## set

### `set`과 `frozenset`의 차이점

set은 해시를 할 수 없지만 frozenset은 해싱이 가능하다.

In [3]:
s = {'a', 'b', 'c'}
hash(s)

TypeError: unhashable type: 'set'

In [5]:
fs = frozenset(['d', 'e', 'f'])
hash(fs)

-5066759644465665399

`set`을 해싱할 수 없는 이유는 `set`은 가변적이기 때문이다. 가변적인 특성 때문에 딕셔너리의 키 또는 다른 `set`의 subset으로 사용될 수 없다.

In [10]:
s.add('d')
print(s)
s.pop()
print(s)
print(dir(s))

{'a', 'b', 'd'}
{'b', 'd'}
['__and__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__iand__', '__init__', '__init_subclass__', '__ior__', '__isub__', '__iter__', '__ixor__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__rand__', '__reduce__', '__reduce_ex__', '__repr__', '__ror__', '__rsub__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__xor__', 'add', 'clear', 'copy', 'difference', 'difference_update', 'discard', 'intersection', 'intersection_update', 'isdisjoint', 'issubset', 'issuperset', 'pop', 'remove', 'symmetric_difference', 'symmetric_difference_update', 'union', 'update']


In [13]:
dic = {
    s: "set's value"
}


TypeError: unhashable type: 'set'

In [14]:
mother_s = {'mother', s}

TypeError: unhashable type: 'set'

반면에 `frozenset`은 집합 내 요소를 변경할 수 없는 대신에 딕셔너리의 키 또는 다른 집합의 하위집합으로 사용 가능하다.

In [11]:
print(dir(fs))
fs.add('g')
fs.pop()

['__and__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__rand__', '__reduce__', '__reduce_ex__', '__repr__', '__ror__', '__rsub__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__xor__', 'copy', 'difference', 'intersection', 'isdisjoint', 'issubset', 'issuperset', 'symmetric_difference', 'union']


AttributeError: 'frozenset' object has no attribute 'add'

In [16]:
dic = {
    fs: "frozenset's value"
}
print(dic)

{frozenset({'e', 'd', 'f'}): "frozenset's value"}


In [17]:
mother_fs = frozenset(['mother', fs])
print(mother_fs)

frozenset({frozenset({'e', 'd', 'f'}), 'mother'})


> 해싱의 대상이 되는 키가 가변적이라면 해시테이블에서 키가 변경되기 전의 hashed result를 탐색할 수 없게 되고 키를 잃어버린 해시 값들이 마구 쌓일것이다. 

In [ ]:
haystack = list(range(1, 1000))
needle = list(range(222, 400))

found_am = set(haystack) & set(needle)
found_in = set(needle).intersection(haystack)

print(found_am)
print(found_in)

`haystack` 리스트 안에 있는 `needle`의 요소가 무엇인지 찾아내는 연산을 할 때 리스트를 순회하는 것보다 교집합 연산을 하는 것이 더 효율적이다.<br>
`set`을 이용한 교집합 연산의 시간복잡도는 `O(min(len(haystack), len(needle)))`이다.

In [41]:
import timeit

setup = '''
haystack = list(range(1, 100))
needle = list(range(22, 40))
'''

print(timeit.timeit('set(haystack) & set(needle)', setup=setup))

code_loop = '''
found = []
for n in needle:
    if n in haystack:
        found.append(n)
'''
print(timeit.timeit(code_loop, setup=setup))


2.569213618000049
7.570099704000313


## literal set

공집합은 리터럴로 표현 할 수 없으므로 `set()`으로 표기해야 한다. `{1,2,3}`과 같은 리터럴 집합 구문은 `set([1,2,3])`처럼 생성자를 호출하는 
것보다 더 빠르고 가독성이 좋다. 생성자를 호출하면 생성자를 가져오기 위해 클래스를 검색하고 리스트를 생성하고 리스트를 생성자에게 전달해야 하므로 
리터럴 집합보다 느리다. 리터럴 집합 구문을 이용할 경우 파이썬은 `BUILD_SET`이라는 바이트코드를 실행한다. <br>
https://docs.python.org/3/library/dis.html#opcode-BUILD_SET

In [38]:
from dis import dis

dis('{1}')
dis('set([1])')

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE
  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


`frozenset`은 별도의 리터럴 구문 없이 생성자를 호출해야 한다.

## 집합 연산 흉내내기

`and`, `or`, `intersection`과 같은 집합 연산은 두 집합 간에서만 가능하고 리스트와 같은 컬렉션에서는 불가능하다.

In [3]:
a = {1, 2, 3}
c = [3, 4, 5]
a & c

TypeError: unsupported operand type(s) for &: 'set' and 'list'

`set`과 `frozenset`에서 사용가능한 메서드 중 하나인 `__and__()`(intersection연산)를 호출할 때 인자로 집합이 아닌 리스트를
넘겨주면 아래와 같은 결과가 리턴된다.

In [2]:
a.__and__(c)

NotImplemented

> **NotImplemented**<br>
Special value which should be returned by the binary special methods (e.g. __eq__(), __lt__(), __add__(), __rsub__(), etc.) to indicate that the operation is not implemented with respect to the other type; may be returned by the in-place binary special methods (e.g. __imul__(), __iand__(), etc.) for the same purpose. Its truth value is true.
<br> Note When a binary (or in-place) method returns NotImplemented the interpreter will try the reflected operation on the other type (or some other fallback, depending on the operator). If all attempts return NotImplemented, the interpreter will raise an appropriate exception. Incorrectly returning NotImplemented will result in a misleading error message or the NotImplemented value being returned to Python code.
See Implementing the arithmetic operations for examples.
<br>Note NotImplementedError and NotImplemented are not interchangeable, even though they have similar names and purposes. See NotImplementedError for details on when to use it.

집합연산을 제공하지 않는 타입의 데이터를 집합연산이 가능하도록 하려면 아래의 메소드를 정의한다.

```python
object.__radd__(self, other)
object.__rsub__(self, other)
object.__rmul__(self, other)
object.__rmatmul__(self, other)
object.__rtruediv__(self, other)
object.__rfloordiv__(self, other)
object.__rmod__(self, other)
object.__rdivmod__(self, other)
object.__rpow__(self, other)
object.__rlshift__(self, other)
object.__rrshift__(self, other)
object.__rand__(self, other)
object.__rxor__(self, other)
object.__ror__(self, other)
```
> These methods are called to implement the binary arithmetic operations (+, -, *, @, /, //, %, divmod(), pow(), **, <<, >>, &, ^, |) with reflected (swapped) operands. These functions are only called if the left operand does not support the corresponding operation [3] and the operands are of different types. [4] For instance, to evaluate the expression x - y, where y is an instance of a class that has an __rsub__() method, y.__rsub__(x) is called if x.__sub__(y) returns NotImplemented.

In [6]:
class SetList(list):
    def __rand__(self, val):
        print('rand called')
        return set(self) & val
    
sl = SetList([3, 4, 5])
print(a & sl)

rand called
{3}


`list`를 상속받은 `SetList`클래스를 정의하고 `__rand__()`함수를 정의했다.<br>
집합이 아닌 피연산자 `sl`를 인자로 넘겨 `a.__and__(sl)`호출하면 `NotImplemented`를 리턴하므로
`sl`의 `__rand__()`메소드를 호출하게 된다.<br>
`__rand__()`는 리스트를 집합으로 변환하여 intersection연산한 결과를 리턴한다.